# Custom Exceptions Examples - Meaningful vs Trivial Usage

## 📋 Overview

This notebook demonstrates meaningful vs trivial usage of custom exceptions in real-world business scenarios.

**Key Learning Goals:**
- Understand when custom exceptions add business value
- Learn proper exception hierarchy design
- See examples of exception handling with logging and recovery
- Understand the difference between technical and business exceptions

## ✅ Meaningful Usage Examples

### 🎯 Example 1: Business Domain Exceptions with Hierarchy

In [ ]:
// Meaningful: Business-focused exception hierarchy with specific contexts

// Base business exception with common properties
public abstract class BusinessException : Exception
{
    public string ErrorCode { get; }
    public string UserMessage { get; }
    public Dictionary<string, object> Context { get; }

    protected BusinessException(
        string errorCode, 
        string userMessage, 
        string technicalMessage,
        Exception innerException = null) 
        : base(technicalMessage, innerException)
    {
        ErrorCode = errorCode;
        UserMessage = userMessage;
        Context = new Dictionary<string, object>();
    }

    public void AddContext(string key, object value)
    {
        Context[key] = value;
    }
}

// E-commerce domain-specific exceptions
public class InsufficientInventoryException : BusinessException
{
    public int ProductId { get; }
    public int RequestedQuantity { get; }
    public int AvailableQuantity { get; }

    public InsufficientInventoryException(
        int productId, 
        int requestedQuantity, 
        int availableQuantity)
        : base(
            "INSUFFICIENT_INVENTORY",
            $"Only {availableQuantity} items available, but {requestedQuantity} requested.",
            $"Inventory check failed for product {productId}: requested={requestedQuantity}, available={availableQuantity}")
    {
        ProductId = productId;
        RequestedQuantity = requestedQuantity;
        AvailableQuantity = availableQuantity;
        
        AddContext("ProductId", productId);
        AddContext("RequestedQuantity", requestedQuantity);
        AddContext("AvailableQuantity", availableQuantity);
        AddContext("Timestamp", DateTime.UtcNow);
    }
}

public class InvalidPaymentMethodException : BusinessException
{
    public string PaymentMethod { get; }
    public decimal Amount { get; }
    public string Currency { get; }

    public InvalidPaymentMethodException(
        string paymentMethod, 
        decimal amount, 
        string currency, 
        string reason)
        : base(
            "INVALID_PAYMENT_METHOD",
            $"Payment method '{paymentMethod}' cannot be used for this transaction.",
            $"Payment validation failed: method={paymentMethod}, amount={amount} {currency}, reason={reason}")
    {
        PaymentMethod = paymentMethod;
        Amount = amount;
        Currency = currency;
        
        AddContext("PaymentMethod", paymentMethod);
        AddContext("Amount", amount);
        AddContext("Currency", currency);
        AddContext("Reason", reason);
    }
}

public class CustomerSuspendedException : BusinessException
{
    public int CustomerId { get; }
    public DateTime SuspendedDate { get; }
    public string SuspensionReason { get; }

    public CustomerSuspendedException(
        int customerId, 
        DateTime suspendedDate, 
        string suspensionReason)
        : base(
            "CUSTOMER_SUSPENDED",
            "Your account has been temporarily suspended. Please contact customer service.",
            $"Customer {customerId} is suspended since {suspendedDate}: {suspensionReason}")
    {
        CustomerId = customerId;
        SuspendedDate = suspendedDate;
        SuspensionReason = suspensionReason;
        
        AddContext("CustomerId", customerId);
        AddContext("SuspendedDate", suspendedDate);
        AddContext("SuspensionReason", suspensionReason);
    }
}

### 🎯 Example 2: Service Integration Exceptions with Recovery

In [ ]:
// Meaningful: Integration exceptions with specific recovery strategies

public abstract class IntegrationException : Exception
{
    public string ServiceName { get; }
    public bool IsRetryable { get; }
    public TimeSpan SuggestedRetryDelay { get; }

    protected IntegrationException(
        string serviceName,
        string message,
        bool isRetryable = false,
        TimeSpan? suggestedRetryDelay = null,
        Exception innerException = null)
        : base(message, innerException)
    {
        ServiceName = serviceName;
        IsRetryable = isRetryable;
        SuggestedRetryDelay = suggestedRetryDelay ?? TimeSpan.FromSeconds(30);
    }
}

public class PaymentGatewayTimeoutException : IntegrationException
{
    public string TransactionId { get; }
    public decimal Amount { get; }

    public PaymentGatewayTimeoutException(
        string transactionId,
        decimal amount,
        TimeSpan timeout)
        : base(
            "PaymentGateway",
            $"Payment gateway timeout for transaction {transactionId} (amount: {amount}) after {timeout.TotalSeconds}s",
            isRetryable: true,
            suggestedRetryDelay: TimeSpan.FromMinutes(2))
    {
        TransactionId = transactionId;
        Amount = amount;
    }
}

public class InventoryServiceUnavailableException : IntegrationException
{
    public List<int> ProductIds { get; }

    public InventoryServiceUnavailableException(
        List<int> productIds,
        string errorDetails)
        : base(
            "InventoryService",
            $"Inventory service unavailable for {productIds.Count} products: {errorDetails}",
            isRetryable: true,
            suggestedRetryDelay: TimeSpan.FromSeconds(45))
    {
        ProductIds = productIds ?? new List<int>();
    }
}

public class EmailDeliveryFailedException : IntegrationException
{
    public string EmailAddress { get; }
    public string EmailType { get; }
    public int AttemptNumber { get; }

    public EmailDeliveryFailedException(
        string emailAddress,
        string emailType,
        int attemptNumber,
        string reason)
        : base(
            "EmailService",
            $"Failed to deliver {emailType} email to {emailAddress} (attempt {attemptNumber}): {reason}",
            isRetryable: attemptNumber < 3,
            suggestedRetryDelay: TimeSpan.FromMinutes(attemptNumber * 5))
    {
        EmailAddress = emailAddress;
        EmailType = emailType;
        AttemptNumber = attemptNumber;
    }
}

### 🎯 Example 3: Exception Handling with Logging and Recovery

In [ ]:
// Meaningful: Exception handling with business logic and recovery

public class OrderService
{
    private readonly ILogger<OrderService> _logger;
    private readonly IInventoryService _inventoryService;
    private readonly IPaymentService _paymentService;
    private readonly IEmailService _emailService;

    public OrderService(
        ILogger<OrderService> logger,
        IInventoryService inventoryService,
        IPaymentService paymentService,
        IEmailService emailService)
    {
        _logger = logger;
        _inventoryService = inventoryService;
        _paymentService = paymentService;
        _emailService = emailService;
    }

    public async Task<OrderResult> ProcessOrderAsync(Order order)
    {
        var result = new OrderResult { OrderId = order.Id };

        try
        {
            // Validate customer status
            await ValidateCustomerAsync(order.CustomerId);
            
            // Check inventory with specific exception handling
            await ValidateInventoryAsync(order);
            
            // Process payment with retry logic
            await ProcessPaymentWithRetryAsync(order);
            
            // Send confirmation email
            await SendConfirmationEmailAsync(order);
            
            result.IsSuccessful = true;
            _logger.LogInformation("Order {OrderId} processed successfully", order.Id);
        }
        catch (CustomerSuspendedException ex)
        {
            // Business exception: Customer cannot place orders
            _logger.LogWarning(ex, "Order attempt by suspended customer {CustomerId}", ex.CustomerId);
            result.ErrorMessage = ex.UserMessage;
            result.ErrorCode = ex.ErrorCode;
            result.RequiresCustomerAction = true;
        }
        catch (InsufficientInventoryException ex)
        {
            // Business exception: Suggest alternatives
            _logger.LogInformation(ex, "Insufficient inventory for order {OrderId}", order.Id);
            result.ErrorMessage = ex.UserMessage;
            result.ErrorCode = ex.ErrorCode;
            result.SuggestedAlternatives = await GetAlternativeProductsAsync(ex.ProductId);
        }
        catch (InvalidPaymentMethodException ex)
        {
            // Business exception: Payment method issue
            _logger.LogWarning(ex, "Invalid payment method for order {OrderId}", order.Id);
            result.ErrorMessage = ex.UserMessage;
            result.ErrorCode = ex.ErrorCode;
            result.RequiresPaymentUpdate = true;
        }
        catch (PaymentGatewayTimeoutException ex) when (ex.IsRetryable)
        {
            // Integration exception: Schedule retry
            _logger.LogError(ex, "Payment gateway timeout for order {OrderId}, scheduling retry", order.Id);
            await ScheduleOrderRetryAsync(order, ex.SuggestedRetryDelay);
            result.ErrorMessage = "Payment processing is temporarily unavailable. We'll retry your order automatically.";
            result.WillRetry = true;
            result.RetryScheduledFor = DateTime.UtcNow.Add(ex.SuggestedRetryDelay);
        }
        catch (EmailDeliveryFailedException ex) when (ex.IsRetryable)
        {
            // Integration exception: Email failed but order succeeded
            _logger.LogWarning(ex, "Email delivery failed for order {OrderId}, will retry", order.Id);
            await ScheduleEmailRetryAsync(order, ex.SuggestedRetryDelay);
            result.IsSuccessful = true; // Order still succeeded
            result.WarningMessage = "Order confirmed, but confirmation email is delayed.";
        }
        catch (IntegrationException ex)
        {
            // General integration exception
            _logger.LogError(ex, "Integration failure during order processing {OrderId}", order.Id);
            result.ErrorMessage = "We're experiencing technical difficulties. Please try again later.";
            result.ErrorCode = "TECHNICAL_ERROR";
            
            if (ex.IsRetryable)
            {
                await ScheduleOrderRetryAsync(order, ex.SuggestedRetryDelay);
                result.WillRetry = true;
            }
        }
        catch (Exception ex)
        {
            // Unexpected exception
            _logger.LogCritical(ex, "Unexpected error processing order {OrderId}", order.Id);
            result.ErrorMessage = "An unexpected error occurred. Our team has been notified.";
            result.ErrorCode = "UNEXPECTED_ERROR";
            
            // Alert monitoring system
            await AlertMonitoringSystemAsync(ex, order);
        }

        return result;
    }

    private async Task ValidateCustomerAsync(int customerId)
    {
        var customer = await GetCustomerAsync(customerId);
        if (customer.Status == CustomerStatus.Suspended)
        {
            throw new CustomerSuspendedException(
                customerId, 
                customer.SuspendedDate.Value, 
                customer.SuspensionReason);
        }
    }

    private async Task ValidateInventoryAsync(Order order)
    {
        foreach (var item in order.Items)
        {
            var availability = await _inventoryService.CheckAvailabilityAsync(item.ProductId);
            if (availability.Available < item.Quantity)
            {
                throw new InsufficientInventoryException(
                    item.ProductId, 
                    item.Quantity, 
                    availability.Available);
            }
        }
    }

    // Additional helper methods...
    private async Task ProcessPaymentWithRetryAsync(Order order) { /* Implementation */ }
    private async Task SendConfirmationEmailAsync(Order order) { /* Implementation */ }
    private async Task ScheduleOrderRetryAsync(Order order, TimeSpan delay) { /* Implementation */ }
    private async Task ScheduleEmailRetryAsync(Order order, TimeSpan delay) { /* Implementation */ }
    private async Task<List<Product>> GetAlternativeProductsAsync(int productId) { return new(); }
    private async Task<Customer> GetCustomerAsync(int customerId) { return new(); }
    private async Task AlertMonitoringSystemAsync(Exception ex, Order order) { /* Implementation */ }
}

public class OrderResult
{
    public int OrderId { get; set; }
    public bool IsSuccessful { get; set; }
    public string ErrorMessage { get; set; }
    public string ErrorCode { get; set; }
    public string WarningMessage { get; set; }
    public bool RequiresCustomerAction { get; set; }
    public bool RequiresPaymentUpdate { get; set; }
    public bool WillRetry { get; set; }
    public DateTime? RetryScheduledFor { get; set; }
    public List<Product> SuggestedAlternatives { get; set; } = new();
}

## ❌ Trivial Usage Examples (What to Avoid)

### 🚫 Example 1: Generic Custom Exception

In [ ]:
// Trivial: Generic exception that doesn't add business value

public class MyCustomException : Exception  // Too generic, no business context
{
    public MyCustomException() : base() { }
    public MyCustomException(string message) : base(message) { }
    public MyCustomException(string message, Exception innerException) : base(message, innerException) { }
    // No additional properties or business logic
}

// Trivial usage - could just use built-in exceptions
public class TrivialService
{
    public void DoSomething(string input)
    {
        if (string.IsNullOrEmpty(input))
        {
            throw new MyCustomException("Input cannot be null");  // Could use ArgumentNullException
        }
        
        if (input.Length > 100)
        {
            throw new MyCustomException("Input too long");  // Could use ArgumentException
        }
    }

    // Trivial catch that doesn't add value
    public void CallDoSomething(string input)
    {
        try
        {
            DoSomething(input);
        }
        catch (MyCustomException ex)
        {
            Console.WriteLine(ex.Message);  // Just printing, no business logic
            throw;  // Re-throwing without adding value
        }
    }
}

### 🚫 Example 2: Wrapper Exception Without Business Logic

In [ ]:
// Trivial: Exception wrapper that doesn't add business value

public class DatabaseException : Exception  // Too generic
{
    public DatabaseException(string message) : base(message) { }
    public DatabaseException(string message, Exception innerException) : base(message, innerException) { }
    // No business context, recovery strategy, or specific handling
}

public class TrivialDataService
{
    // Trivial exception wrapping without adding value
    public User GetUser(int id)
    {
        try
        {
            // Database call
            return database.GetUser(id);
        }
        catch (SqlException ex)
        {
            // Just wrapping without adding business context
            throw new DatabaseException("Database error occurred", ex);
        }
    }

    // Trivial handling that doesn't differentiate
    public void ProcessUser(int userId)
    {
        try
        {
            var user = GetUser(userId);
            // Process user...
        }
        catch (DatabaseException ex)
        {
            // Generic handling - no specific business logic
            Console.WriteLine("Something went wrong with database");
            // No recovery, no business-specific action
        }
    }
}

### 🚫 Example 3: Exception Without Proper Context

In [ ]:
// Trivial: Exception without business context or recovery information

public class ValidationException : Exception  // Too generic name
{
    public ValidationException(string message) : base(message) { }
    // No information about what failed validation or how to fix it
}

public class TrivialValidator
{
    // Trivial validation that provides no business context
    public void ValidateEmail(string email)
    {
        if (!email.Contains("@"))
        {
            throw new ValidationException("Invalid email");  // No specifics
        }
    }

    public void ValidateAge(int age)
    {
        if (age < 0)
        {
            throw new ValidationException("Invalid age");  // No business rules
        }
    }

    // Trivial handling without business logic
    public bool TryValidateUser(string email, int age)
    {
        try
        {
            ValidateEmail(email);
            ValidateAge(age);
            return true;
        }
        catch (ValidationException)
        {
            return false;  // No information about what failed
        }
    }
}

## 💡 Key Principles for Meaningful Exception Usage

### ✅ What Makes Exception Usage Meaningful:

1. **Business Context**
   - Exceptions represent specific business error conditions
   - Include relevant business data (IDs, amounts, etc.)
   - Provide user-friendly messages separate from technical details

2. **Specific Handling Strategies**
   - Different exception types require different handling
   - Recovery mechanisms built into exception design
   - Retry logic based on exception properties

3. **Proper Logging and Monitoring**
   - Structured logging with business context
   - Different log levels based on exception severity
   - Integration with monitoring and alerting systems

4. **Exception Hierarchy**
   - Well-designed inheritance hierarchy
   - Base classes with common properties and behavior
   - Specific derived classes for different scenarios

### ❌ What Makes Exception Usage Trivial:

1. **Generic Exceptions**
   - Exceptions with generic names like 'CustomException'
   - No business context or specific information
   - Could be replaced with built-in exception types

2. **Simple Wrappers**
   - Just wrapping other exceptions without adding value
   - No additional business information or context
   - No specific handling strategy

3. **Poor Exception Handling**
   - Catching exceptions just to re-throw them
   - Generic handling that doesn't differentiate exception types
   - No recovery logic or business-specific actions